In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

from assistant.tools import fetch_github_repo
from assistant.agents import Agent
from assistant.prompts import github_cleaner_agent_system_prompt

In [2]:
github_cleaner_agent_system_prompt()

{'role': 'system',
 'content': 'You are an expert at reading code and also developing programs. You will be given some data from a github repo. \n    Primary Objectives: \n    1. Convert any notebook code or other markdown (e.g. .ipynb) into more readable python\n    For example: ```{\n   "cell_type": "code",\n   "execution_count": 10,\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "from dotenv import load_dotenv\\n",\n    "from openai import OpenAI\\n",\n    "from tenacity import retry, wait_random_exponential, stop_after_attempt"\n   ]\n  }```\n    Should become: ``` from dotenv import load_dotenv\nfrom openai import OpenAI\nfrom tenacity import retry, wait_random_exponential, stop_after_attempt```\n    3. Then, give me output in the following format:\n    \n    \n     ## Folder Structure\n     (insert folder structure here)\n\n     ## File contents\n     file1.py\n     ```file1.py contents```\n\n     file2.ipynb\n     ```file2.ipynb contents```\n    \n'}

In [3]:
load_dotenv()

GPT_MODEL = 'gpt-3.5-turbo'

client = OpenAI()



In [7]:
messages = [
    {'role': 'user', 'content': 'Just saying hello!'}
]

response = client.chat.completions.create(
    model=GPT_MODEL,
    messages=messages,
)


In [8]:
response

ChatCompletion(id='chatcmpl-9i4zRu6Btq006QpOCBxSp47HlLFMP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1720292641, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=11, total_tokens=20))

In [ ]:
github_cleaner_agent = Agent()

github_cleaner_agent.memory.append(github_cleaner_agent_system_prompt())

github_cleaner_agent.tools.append(
    {
        'type': 'function',
        'function': {
            'name': 'placeholder_tool',
            'description': 'Placeholder tool not to be used',
            'parameters': {
                'type': 'object',
                'properties': {
                    'placeholder_property': {
                        'type': 'string',
                        'description': 'Placeholder property',
                    },
                    'placeholder_property_2': {
                        'type': 'integer',
                        'description': 'Placeholder property 2'
                    },
                },
                'required': ['placeholder_property', 'placeholder_property_2']
            }
        }
    }
)


repo_data = fetch_github_repo("https://github.com/cetyz/coding-ai")

github_response = github_cleaner_agent.invoke(str(repo_data))
print(github_response)

# while True:
#     user_input = input('User Message:')
#     if user_input == 'exit':
#         break
#     response = agent.invoke(user_input)
#     print('Assistant:', response)
#     print()

In [ ]:
github_analyst_agent = Agent()

github_analyst_agent.memory.append({
        'role': 'system',
        'content': f'''You are an expert at reading code and also developing programs. You will be given some data from a github repo. 

    Here is the github data: {github_response}

    Primary Objectives: 
    1. Understand the repository and answer any questions the user may have
    2. Think step by step how to answer this question
    3. Give the final answer

    Use the following framework for thinking and answering:
    
    Summary of user question: [summary]
    Possible solutions to question: [possible solutions]
    Given the current repo/code, shortlist solutions further: [shortlisted solutions]
    Final answer: [choose solution, followed by reasoning]
    Implementation of code: [```insert code or changes```]
    Explanation of implementation: [explain written/edited code]
     
'''
    })

github_analyst_agent.tools.append(
    {
        'type': 'function',
        'function': {
            'name': 'placeholder_tool',
            'description': 'Placeholder tool not to be used',
            'parameters': {
                'type': 'object',
                'properties': {
                    'placeholder_property': {
                        'type': 'string',
                        'description': 'Placeholder property',
                    },
                    'placeholder_property_2': {
                        'type': 'integer',
                        'description': 'Placeholder property 2'
                    },
                },
                'required': ['placeholder_property', 'placeholder_property_2']
            }
        }
    }
)




analysis_response = github_analyst_agent.invoke('I want to make this into an app using streamlit. How can I do so and what is the recommended folder structure?')
print(analysis_response)

In [ ]:


# Example usage:
repo_data = str(fetch_github_repo("https://github.com/cetyz/coding-ai"))
print(repo_data)